In [1]:
##Robs summariser of RC8 regoinal Contributor tables
import pandas as pd
import numpy as np
import os

#mainPath = 'P:\projects\RC9_ResultsSets\RegContribTables'
#mainPreDevPath = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth'
#localBaselinePath = r'E:\GBRFScenarios\RC10_BASELINE_REGCONTRIBS'
scenarioRegContribsPath = r'E:\GBRFScenarios\Results'
outPath = r'E:\GBRFScenarios\Results\WarmupSummaries'
summariesOutFolder = 'SummariesWithProcess'
#baseFolderEnd = 'BASE_RC10'
#changeFolderEnd = 'CHANGE_RC10'
#predevFolderEnd = 'PREDEV_RC10'
regContributorFileName = 'RegContributorDataGrid.csv'
#outputCSVEnd = 'RegToOutlet_Alt.csv'
outputCSVEnd = 'RegToOutlet.csv'
baselineScenarioName = 'Baseline'
#changeScenarioName = 'Change'
predevScenarioName = 'PreDev'


pathToContstitsToGrpFile = r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\constituentsToGroups.csv'

#Must be utilised in regContributors that do NOT include nesting structures
alternativeRegionsCSV = {'FI':r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\FI_ReportingRegions_Details.csv',
                        'BU':r'\\athenasmb\pdreef\RC10_RC2019\RC10_RC2019_ResultsSets_PointOfTruth\BU_ReportingRegions_Details.csv'}

altRegLinker = 'Catchmt'
standardRegName = 'RepReg'
altRegColName = 'AUSGMCAS'
origLinker = 'ModelElement'


annLoadToStream = 'AnnLoadToStreamKG'
annLoadToExport = 'AnnLoadToExportKG'

#regionIDs = ['BM', 'FI', 'MW']

regionIDs = {'MW':['MW_GBRF_DIN_MUD_CtoB_WARMUP','MW_GBRF_DIN_MUD_DtoB_WARMUP']}

standardScenariosDict = {baselineScenarioName:localBaselinePath, predevScenarioName:mainPreDevPath}
#scenarioToFolderDict = {baselineScenarioName:baseFolderEnd, changeScenarioName:changeFolderEnd, predevScenarioName:predevFolderEnd}


NameError: name 'localBaselinePath' is not defined

In [2]:
if not os.path.exists(outPath):
    os.makedirs(outPath)
    print("Made dir: " + outPath)

Made dir: E:\GBRFScenarios\Results\WarmupSummaries


In [3]:
if not os.path.exists(outPath + '\\' + summariesOutFolder):
    os.makedirs(outPath + '\\' + summariesOutFolder)
    print("Made dir: " + outPath + '\\' + summariesOutFolder)

Made dir: E:\GBRFScenarios\Results\WarmupSummaries\SummariesWithProcess


In [4]:
def getPreDevPathInfo(regionIDString, scenarioString):
    fileIn = scenarioRegContribsPath + '\\' + regionIDString + '\\MW_PREDEV_RC10_WARMUP_RegContributorDataGrid.csv'
    return fileIn

In [5]:
def getBaselinePathInfo(regionIDString, scenarioString):
    fileIn = scenarioRegContribsPath + '\\' + regionIDString + '\\MW_BASEGBRF_RC10_WARMUP_RegContributorDataGrid.csv'
    return fileIn

In [6]:
def getScenarioPathInfo(regionIDString, scenarioString):
    fileIn = scenarioRegContribsPath + '\\' + regionIDString + '\\' + scenarioString + '_' + regContributorFileName
    return fileIn

In [7]:
def getOutPathInfo(regionIDString, scenarioString):
    fileOut = outPath + '\\' + regionIDString + '_' + scenarioString + '_' + outputCSVEnd
    return fileOut

In [8]:
def produceSummaryFromRegContrib(regionIDString, scenarioName, theFileIn, theFileOut):
    
    #theFileOut = getOutPathInfo(regionIDString, scenarioName)
    
    print("Processing " + theFileIn + " Scenario: " + scenarioName)
    
    rawcontrib = pd.read_csv(theFileIn)
    
    rawcontrib['SCENARIO'] = scenarioName
    rawcontrib[annLoadToStream] = rawcontrib['LoadToStream (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib[annLoadToExport] = rawcontrib['LoadToRegExport (kg)'].div(rawcontrib['Num_Days']).mul(365.25)
    rawcontrib['AreaHA'] = rawcontrib['AreaM2'].div(10000)
    
    if regionIDString in alternativeRegionsCSV:
        print("Reading alternative regionalistaion for " + regionIDString)
        altDetsDF = pd.read_csv(alternativeRegionsCSV[regionIDString])
        combinedDF = pd.merge(rawcontrib, altDetsDF, how='left', left_on=[origLinker], right_on = [altRegLinker])
        #Transfer vals
        combinedDF['Rep_Region'] = combinedDF[altRegColName]
        #Drop cols
        #where 1 is the axis number (0 for rows and 1 for columns.)
        combinedDF = combinedDF.drop(altRegColName, 1)
        combinedDF = combinedDF.drop(standardRegName, 1)
        combinedDF = combinedDF.drop(altRegLinker, 1)
        rawcontrib = combinedDF
        
    
    ### Can keep reporting region in the area table, as this is our Non-Nested notebook
    catchFuArea = pd.DataFrame(rawcontrib.groupby(['Rep_Region','ModelElement','FU']).agg({'AreaHA':'first'})).reset_index()
    regFuArea = pd.DataFrame(catchFuArea.groupby(['Rep_Region','FU']).agg({'AreaHA':'sum'})).reset_index()
    
    #regLuseSummary = pd.DataFrame(rawcontrib[rawcontrib['Constituent'].isin(selectedConstituents)].groupby(['Rep_Region','Constituent','FU','Process']).agg({'AnnLoadToStreamKG':'sum','AnnLoadToExportKG':'sum'})).reset_index()
    regLuseSummary = pd.DataFrame(rawcontrib.groupby(['SCENARIO','Rep_Region','Constituent','FU','Process']).agg({annLoadToStream:'sum',annLoadToExport:'sum'})).reset_index()
    
    ### Join/merge with Regional Areas
    regLuseSumPlusArea = pd.merge(regLuseSummary, regFuArea, how='left', left_on=['Rep_Region','FU'], right_on = ['Rep_Region','FU'])
    
    regLuseSumPlusArea.to_csv(theFileOut, index=False)
    
    print("Saved " + theFileOut)
    
    return

    

In [9]:
#Do standard scenarios first

for theReg in regionIDs:
    
    print("Doing: " + theReg + " " + baselineScenarioName)
    baseInFile = getBaselinePathInfo(theReg, baselineScenarioName)
    baseOutFile = getOutPathInfo(theReg, baselineScenarioName)
    produceSummaryFromRegContrib(theReg, baselineScenarioName, baseInFile, baseOutFile)
    
    print("Doing: " + theReg + " " + predevScenarioName)
    predevInFile = getPreDevPathInfo(theReg, predevScenarioName)
    predevOutFile = getOutPathInfo(theReg, predevScenarioName)
    produceSummaryFromRegContrib(theReg, predevScenarioName, predevInFile, predevOutFile)
    
    #Now scenarios
    
    for theScen in regionIDs[theReg]:
        
        print("Doing: " + theReg + " " + theScen)
        scenInFile = getScenarioPathInfo(theReg, theScen)
        scenOutFile = getOutPathInfo(theReg, theScen)
        produceSummaryFromRegContrib(theReg, theScen, scenInFile, scenOutFile)


print("Finished creating individual summary files")

Doing: MW Baseline
Processing E:\GBRFScenarios\Results\MW\MW_BASEGBRF_RC10_WARMUP_RegContributorDataGrid.csv Scenario: Baseline
Saved E:\GBRFScenarios\Results\WarmupSummaries\MW_Baseline_RegToOutlet.csv
Doing: MW PreDev
Processing E:\GBRFScenarios\Results\MW\MW_PREDEV_RC10_WARMUP_RegContributorDataGrid.csv Scenario: PreDev
Saved E:\GBRFScenarios\Results\WarmupSummaries\MW_PreDev_RegToOutlet.csv
Doing: MW MW_GBRF_DIN_MUD_CtoB_WARMUP
Processing E:\GBRFScenarios\Results\MW\MW_GBRF_DIN_MUD_CtoB_WARMUP_RegContributorDataGrid.csv Scenario: MW_GBRF_DIN_MUD_CtoB_WARMUP
Saved E:\GBRFScenarios\Results\WarmupSummaries\MW_MW_GBRF_DIN_MUD_CtoB_WARMUP_RegToOutlet.csv
Doing: MW MW_GBRF_DIN_MUD_DtoB_WARMUP
Processing E:\GBRFScenarios\Results\MW\MW_GBRF_DIN_MUD_DtoB_WARMUP_RegContributorDataGrid.csv Scenario: MW_GBRF_DIN_MUD_DtoB_WARMUP
Saved E:\GBRFScenarios\Results\WarmupSummaries\MW_MW_GBRF_DIN_MUD_DtoB_WARMUP_RegToOutlet.csv
Finished creating individual summary files


In [10]:
#Build relationships of constituent names to groups
#tssGrp = 'TSS'
#tnGrp = 'TN'
#tpGrp = 'TP'
#ps2Grp = 'PSII'
#nonps2Grp = 'non-PSII'
##Nope, will read in a CSV to DataFrame instead, thus joing will 'drop' flow from this comparison
#constToGrp = {'Sediment - Fine':tssGrp}

constToGrp = pd.read_csv(pathToContstitsToGrpFile)

baseSupplyStr = 'Base_Supply_KG'
baseExportStr = 'Base_Export_KG'
changeSupplyStr = 'Change_Supply_KG'
changeExportStr = 'Change_Export_KG'
predevSupplyStr = 'PreDev_Supply_KG'
predevExportStr = 'PreDev_Export_KG'
repRegStr = 'Rep_Region'
sourceRegStr = 'Source Region'
constituentStr = 'Constituent'
fuStr = 'FU'
groupStr = 'GROUP'
modelRegStr = 'MODELREG'
scenarioStr = 'SCENARIO'
percRedStr = 'PercReduction'
regTotalStr = 'REGIONAL TOTAL'
processStr = 'Process'

#allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr])
#allExportsAndReductions = pd.DataFrame()


In [11]:
#So now we would be reading in known summary files

def calculateReductions(thisRegID, thisScenario):
    
    theBaseSummaryFile = getOutPathInfo(thisRegID, baselineScenarioName)
    theChangeSummaryFile = getOutPathInfo(thisRegID, thisScenario)
    thePreDevSummaryFile = getOutPathInfo(thisRegID, predevScenarioName)
    
    #print("Reading: " + theBaseSummaryFile)
    baseSummaryDF = pd.read_csv(theBaseSummaryFile)
    #print("Reading: " + theChangeSummaryFile)
    changeSummaryDF = pd.read_csv(theChangeSummaryFile)
    #print("Reading: " + thePreDevSummaryFile)
    predevSummaryDF = pd.read_csv(thePreDevSummaryFile)
    
    #inner drops out those that don't merge, left keep them
    baseTotals = pd.merge(baseSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum', 'AreaHA':'first'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    baseTotals = baseTotals.rename(columns={repRegStr:sourceRegStr,'AreaHA':'AREA_HA',annLoadToStream:baseSupplyStr, annLoadToExport:baseExportStr})
    baseTotals = baseTotals[[sourceRegStr, groupStr, constituentStr, fuStr, processStr, 'AREA_HA', baseSupplyStr, baseExportStr]]
    #baseTotals.head(40)
    
    predevTotals = pd.merge(predevSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT']).reset_index()
    predevTotals = predevTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:predevSupplyStr, annLoadToExport:predevExportStr})
    predevTotals = predevTotals[[sourceRegStr, constituentStr, fuStr, processStr, predevSupplyStr, predevExportStr]]
    
    changeTotals = pd.merge(changeSummaryDF.groupby([repRegStr,constituentStr,fuStr,processStr]).agg({annLoadToStream:'sum', annLoadToExport:'sum'}).reset_index(), constToGrp, how='inner', left_on=[constituentStr], right_on = ['CONSTITUENT'])
    changeTotals = changeTotals.rename(columns={repRegStr:sourceRegStr,annLoadToStream:changeSupplyStr, annLoadToExport:changeExportStr})
    changeTotals = changeTotals[[sourceRegStr, constituentStr, fuStr, processStr, changeSupplyStr, changeExportStr]]
    #changeTotals.head(8)
    
    #, columns=['Rep_Region', 'GROUP', 'Constituent', 'FU', 'AREA_HA', 'Base_Supply_KG', 'Base_Export_KG', 'PreDev_Supply_KG', 'PreDev_Export_KG', 'Change_Supply_KG', 'Change_Export_KG']
    anthroRegByLuse = pd.merge(pd.merge(baseTotals, predevTotals, how='left', left_on=[sourceRegStr,constituentStr,fuStr,processStr],
                                        right_on = [sourceRegStr,constituentStr,fuStr,processStr]).reset_index(), changeTotals, how='left',
                               left_on=[sourceRegStr,constituentStr,fuStr,processStr],
                               right_on = [sourceRegStr,constituentStr,fuStr,processStr])
    
    anthroRegByLuse[scenarioStr] = thisScenario
    #anthroRegByLuse.head(8)
    
    theFileOut = outPath + '\\' + summariesOutFolder + '\\' + thisRegID + '_' + thisScenario + '_AnthroRegByLanduseProcess.csv'
    anthroRegByLuse.to_csv(theFileOut, index=False)
    #print("Saved " + theFileOut)
    
    regExports = anthroRegByLuse.groupby([sourceRegStr, scenarioStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', changeExportStr:'sum'})
    
    regExports[modelRegStr] = thisRegID
    regExports[percRedStr] = (regExports[baseExportStr] - regExports[changeExportStr]).div(regExports[baseExportStr] - regExports[predevExportStr]).mul(100)
    regExports = regExports.reset_index()
    regExports = regExports[[modelRegStr, sourceRegStr, scenarioStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr]]
    #regExports.head(15)
    
    modelRegExports = regExports.groupby([modelRegStr, scenarioStr, constituentStr, groupStr]).agg({predevExportStr:'sum', baseExportStr:'sum', changeExportStr:'sum'})
    modelRegExports[sourceRegStr] = regTotalStr
    modelRegExports[percRedStr] = (modelRegExports[baseExportStr] - modelRegExports[changeExportStr]).div(modelRegExports[baseExportStr] - modelRegExports[predevExportStr]).mul(100)
    modelRegExports = modelRegExports.reset_index()
    modelRegExports = modelRegExports[[modelRegStr, sourceRegStr, scenarioStr, constituentStr, groupStr, predevExportStr, baseExportStr, changeExportStr, percRedStr]]
    ##modelRegExports.head(15)
    regExports = regExports.append(modelRegExports, ignore_index=True)
    
    return regExports


In [12]:
allExportsAndReductions = pd.DataFrame(columns=[modelRegStr, sourceRegStr, scenarioStr, constituentStr, groupStr,
                                                predevExportStr, baseExportStr, changeExportStr, percRedStr])

for theReg in regionIDs:
    for scen in regionIDs[theReg]:
        
        theseExports = calculateReductions(theReg, scen)
        allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)


# theseExports = calculateReductions('Burdekin', 'BU')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Burnett Mary', 'BM')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Cape York', 'CY')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Fitzroy', 'FI')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Mackay Whitsunday', 'MW')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

# theseExports = calculateReductions('Wet Tropics', 'WT')
# allExportsAndReductions = allExportsAndReductions.append(theseExports, ignore_index=True)

reqVals = ['TSS', 'TN', 'TP']

allExportsAndReductions = allExportsAndReductions.loc[allExportsAndReductions[groupStr].isin(reqVals)]

theFileNameOut = outPath + '\\' + summariesOutFolder + '\\GBRF_AllExportsAndReductions_FromProcess.csv'
allExportsAndReductions.to_csv(theFileNameOut, index=False)
allExportsAndReductions.head(20)
#print(allExportsAndReductions)

,MODELREG,Source Region,SCENARIO,Constituent,GROUP,PreDev_Export_KG,Base_Export_KG,Change_Export_KG,PercReduction
0,MW,OConnel,MW_GBRF_DIN_MUD_CtoB_WARMUP,N_DIN,TN,6.337760e+04,2.469350e+05,2.469350e+05,0.000000
1,MW,OConnel,MW_GBRF_DIN_MUD_CtoB_WARMUP,N_DON,TN,1.317339e+05,3.462072e+05,3.462072e+05,0.000000
2,MW,OConnel,MW_GBRF_DIN_MUD_CtoB_WARMUP,N_Particulate,TN,3.345611e+05,9.631116e+05,9.631116e+05,0.000000
3,MW,OConnel,MW_GBRF_DIN_MUD_CtoB_WARMUP,P_DOP,TP,4.501093e+03,1.459564e+04,1.459564e+04,0.000000
4,MW,OConnel,MW_GBRF_DIN_MUD_CtoB_WARMUP,P_FRP,TP,1.554537e+04,5.509354e+04,5.509354e+04,0.000000
5,MW,OConnel,MW_GBRF_DIN_MUD_CtoB_WARMUP,P_Particulate,TP,1.210608e+05,3.178659e+05,3.178659e+05,0.000000
6,MW,OConnel,MW_GBRF_DIN_MUD_CtoB_WARMUP,Sediment - Fine,TSS,6.129148e+07,2.120414e+08,2.120414e+08,0.000000
7,MW,Pioneer,MW_GBRF_DIN_MUD_CtoB_WARMUP,N_DIN,TN,4.363330e+04,2.470970e+05,2.470970e+05,0.000000
8,MW,Pioneer,MW_GBRF_DIN_MUD_CtoB_WARMUP,N_DON,TN,7.955999e+04,2.178635e+05,2.178635e+05,0.000000
9,MW,Pioneer,MW_GBRF_DIN_MUD_CtoB_WARMUP,N_Particulate,TN,2.093677e+05,5.521989e+05,5.521989e+05,0.000000
